# Wikipedia Search
This program allows you to search for wikipedia articles and then rate them after reading them.
Once you quit the program, it tells you the average of your ratings. It stores the ratings in a database so that the information persists across sessions 😋.

In [2]:
#!pip install wikipedia
import wikipedia
from emoji import emojize
import sqlite3
from IPython.display import display, Markdown
from typing import List, Optional

conn: sqlite3.Connection = sqlite3.connect('wikipedia_ratings.db')
cursor: sqlite3.Cursor = conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS ratings
             (articleTitle text, rating integer)
''')

def rate(articleTitle: str, rating: int) -> None:
    cursor.execute('INSERT INTO ratings (articleTitle, rating) VALUES (?, ?)', (articleTitle, rating))
    conn.commit()

def getAvgRating() -> Optional[float]:
    cursor.execute('SELECT AVG(rating) FROM ratings')
    avg_rating: Optional[float] = cursor.fetchone()[0]
    return avg_rating

def ynInput(prompt: str) -> bool:
    return input(prompt + " [y/n]").startswith("y")

def rangedInput(prompt: str, minVal: int, maxVal: int) -> int:
    inp: str = input(prompt)
    while True:
        if inp.isdecimal():
            inp_int: int = int(inp)
            if minVal <= inp_int <= maxVal:
                return inp_int
        inp = input(f"Invalid input. Please enter a number between {minVal} and {maxVal}.")

try:
    while True:
        user_inp: str = input("What Wikipedia page would you like to open?")
        if user_inp.strip() == "":
            raise KeyboardInterrupt()
        
        results: List[wikipedia.WikipediaPage] = wikipedia.search(user_inp)
        if len(results) == 0:
            print("No such article was found.")
            continue
        
        try:
            summary: str = wikipedia.summary(results[0])
            print("Here is a summary of the article:")
            display(Markdown(summary))
        except wikipedia.PageError:
            print("No such article was found.")
            continue
        
        read_full_article: bool = ynInput("Would you like to read the full article?")
        if read_full_article:
            print(f"Ok here's the link {results[0].url}")
            
            rating: int = rangedInput("Please rate this article w/ a number from 1-5 (don't do a SQL injection please)", 1, 5)
            rate(user_inp, rating)
            print(f"You gave a rating of ", end="")
            
            for _ in range(rating):
                print(emojize(":star:"), end="")
except KeyboardInterrupt:
    avg_rating: Optional[float] = getAvgRating()
    print(f"The average rating you give is {avg_rating}")
    conn.close()


The average rating you give is 4.0


In [ ]:
import inspect 
from wikipedia import search

print("Are you curious about how the python wikipedia library's search function works? Well I sure am!")
inspect.getsource(search)